# Differentiable Physics

## Forward Integration of Newton's Equations of Motion

Given a particle with:

- position $x$ 
- mass $m$ 
- velocity $v$ = $\frac{dx}{dt}$ 
- acceleration $a$ = $\frac{dv}{dt}$ = $\frac{d^2x}{dt^2}$
- force $F$ 

For a simple example, we apply the second motion $ F = m \frac{d^2 x}{dt^2} $ along with Hooke's Law $F(x) = -k x$. This results in the following differential equation:

$$ m a = -k x \\
\frac{d^2x}{dt^2} = \frac{-k}{m} x$$

In order to solve such an ODE numerically, we discretize the time $t$ into steps $\Delta t$. Therefore we add a time index $i$ to the variables $x$, $v$ and $a$:

$$ x_i = x(t_i) \\
v_i = v(t_i) \\
a_i = a(t_i) $$

$$ \Delta t = t_{i+1} - t_i $$


We can use the taylor expansion to approximate the positions:

$$ x_{i+1} = x_i + \Delta t \frac{dx}{dt} + \frac{\Delta t^2}{2} \frac{d^2x}{dt^2} + \mathcal{O}(\Delta t^3) $$

And the velocities:

$$ v_{i+1} = v_i + \Delta t \frac{dv}{dt} + \mathcal{O}(\Delta t^2) $$

This taylor series builds the foundation for many numerical integrators such as the Euler method, the Runge-Kutta method or the Leapfrog method.

We will focus on the velocity verlet method:

$$ x_{i+1} = x_i + v_i \Delta t + \frac{1}{2} a_i \Delta t^2 \\
v_{i+1} = v_i + \frac{1}{2} (a_i + a_{i+1}) \Delta t $$

## Differentiable Physics

In order to make the integration of the equations of motion differentiable, we need to find the gradient of the solver with respect to its input. Lets denote the state of system at point $i$ as $s_i = (x_i, v_i)$. The solver $P$ is a function that takes the state $s_i$ and returns the state $s_{i+1}$:

$$ s_{i+1} = P(s_i) $$

which allows us to find the state of the system at any point in time with an initial state $s_0$:

$$ s_{n} = P_n \circ P_{n-1} \circ \dots \circ P_1(s_0) $$



A common task is given a desired enstate $s^*_n$ find $s_0$ such that $||s_n - s^*_n||^2$ is minimal. In theory we could simply use the inverse of the solver $P^{-1}$ to find $s_0$:

$$ s_0 = P^{-1}_1 \circ P^{-1}_2 \circ \dots \circ P^{-1}_n(s^*_n) $$

In practice however, this is oftentimes not possible since inverting the solver $P$ can come at great costs or its not even possible. **Why not?**

However we can *always* find the gradient of the solver $P$ with respect to its input:

$$\frac{\partial P}{\partial s_i} $$

This can become useful to propate the loss with respect to $s_0$, which is defined as:

$$ L = || \left( P_n \circ P_{n-1} \circ \dots \circ P_1(s_0) \right) - s^*_n ||^2 $$

backwards through the solvers:

$$ \frac{\partial L}{\partial s_0} = \frac{\partial L}{\partial s_n} \frac{\partial s_n}{\partial s_{n-1}} \dots \frac{\partial s_1}{\partial s_0} = \frac{\partial L}{\partial s_n} \frac{\partial P_n}{\partial s_{n-1}} \dots \frac{\partial P_1}{\partial s_0} $$

This is now a minimization problem that can be solved with gradient descent or any other optimization algorithm.


## Differentiable Newtons Equations

Let us now apply the principle of differentiable physics to the equations of motion. We start with the forward integration of the equations of motion:

$$ x_{i+1} = x_i + v_i \Delta t + \frac{1}{2} a_i \Delta t^2 \\ 
v_{i+1} = v_i + \frac{1}{2} (a_i + a_{i+1}) \Delta t $$

Let us assume we have force which is only dependent on the position $F(x_i)$. The integration scheme can also be written as:

$$ \begin{bmatrix} x_{i+1} \\ v_{i+1} \end{bmatrix} = \begin{bmatrix} x_i \\ v_i \end{bmatrix} + \Delta t \begin{bmatrix} v_i \\ a_i \end{bmatrix} + \frac{1}{2} \Delta t^2 \begin{bmatrix} 0 \\ a_i \end{bmatrix} $$

This allows us to express the numerical as function $P$ which takes the state $s_i$ or $\begin{bmatrix} x_{i} \\ v_{i} \end{bmatrix}$ and returns the state $s_{i+1}$:

$$ \begin{bmatrix} x_{i+1} \\ v_{i+1} \end{bmatrix} = P \left( \begin{bmatrix} x_{i} \\ v_{i} \end{bmatrix} \right) = \begin{bmatrix} x_{i} \\ v_{i} \end{bmatrix} + \Delta t \begin{bmatrix} v_i \\ a_i \end{bmatrix} + \frac{1}{2} \Delta t^2 \begin{bmatrix} 0 \\ a_i \end{bmatrix} $$


We can now find the gradient of the solver $P$ with respect to its input $s_i$:

$$ \frac{\partial P(s_i)}{\partial s_i} = \begin{bmatrix} \frac{\partial P(s_i)}{\partial x_i} & \frac{\partial P(s_i)}{\partial v_i} \end{bmatrix}

= \begin{bmatrix} 1 + 0.5 \Delta t ^2 \frac{\partial F(x_i)}{2m x_i } & \Delta t \\ \Delta t \left( \frac{\partial F(x_i)}{2m x_i}  + \frac{\partial F(x_{i+1})}{2m x_{i}} \right) & 1 \end{bmatrix}$$

Let us now program this in python. 

In [4]:
import numpy as np
from typing import Callable

class Verlet: 

    def __init__(self, F : Callable, dFdx : Callable) -> None:
        self.F = F
        self.dFdx = dFdx

    def forward(self, x : np.ndarray, v : np.ndarray, dt : float) -> tuple[np.ndarray, np.ndarray]:
        x_fwd = x + v * dt + 0.5 * self.F(x) * dt^2
        v_fwd = v + 0.5 * (self.F(x) + self.F(x_fwd)) * dt

        # cache for backward pass
        self.cache_x = x
        self.cache_v = v
        self.cache_x_fwd = x_fwd

        return x_fwd, v_fwd
    
    def backward(self, grad_x : np.ndarray, grad_v : np.ndarray, dt : float) -> tuple[np.ndarray, np.ndarray]:
        
        grad_x 
        pass




In [ ]:
n = 100

# Desired final particle position and velocity 
x_n_star = np.array([3, 3])
v_n_star = np.array([0, 0])

def loss(approximation : np.ndarray, desired : np.ndarray) -> float:
    return np.linalg.norm(approximation - desired)

# Force function (gravity)
def F(x : np.ndarray) -> np.ndarray:
    return np.array([0, -9.81])

# Derivative of the force function (gravity)
def dFdx(x : np.ndarray) -> np.ndarray:
    return np.array([[0, 0], [0, 0]])

# Guess initial particle position and velocity
x_0 = np.array([0, 0])
v_0 = np.array([0, 0])

# Initialize Verlet integrator
verlet = Verlet(F, dFdx)

for i in range(n):
    x, v = verlet.forward(x_0, v_0, 0.1)
    x_0 = x
    v_0 = v
